In [1]:
from tensorflow import keras as K
from tensorflow.keras import applications

In [17]:
# import libraries
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential,Input,Model

import numpy as np
import matplotlib.pyplot as plt

from PIL import ImageEnhance, ImageFilter
import random
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, save_img

In [18]:
from PIL import Image, ImageOps

# Resize Images and Add Padding

In [15]:
# Desired target size for all images
target_size = (416, 416)
padding_color = (0, 0, 0) #black

# Function to resize and add padding 
def resize_pad_images(parent_folder, target_size, padding_color):
    # Iterate through each class folder
    for class_folder in os.listdir(parent_folder):
        class_folder_path = os.path.join(parent_folder, class_folder)

        # Check if it's a directory
        if os.path.isdir(class_folder_path):
            print(f"Processing folder: {class_folder_path}")

            # Iterate over each image file in the class folder
            for img_name in os.listdir(class_folder_path):
                img_path = os.path.join(class_folder_path, img_name)

                if img_name.endswith(('.png', '.jpg', '.jpeg')):  # Check for valid image files
                    print(f"Processing image: {img_path}")

                    img = Image.open(img_path)

                    # Resize while keeping the aspect ratio
                    img.thumbnail(target_size, Image.Resampling.LANCZOS)

                    # Calculate padding to center the image
                    delta_w = target_size[0] - img.size[0]
                    delta_h = target_size[1] - img.size[1]
                    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))

                     # Add padding
                    img_padded = ImageOps.expand(img, padding, fill=padding_color)

                    # Save the resized and padded image
                    img_padded.save(img_path)
                    print(f"Saved image: {img_path}")


In [10]:
# Paths to the train, test, and valid folders
train_folder = '../data/train/'
test_folder = '../data/test/'
valid_folder = '../data/valid/'

In [19]:
# Apply resizing and padding to all folders
print("Processing train folder...")
resize_pad_images(train_folder, target_size, padding_color)

print("Processing test folder...")
resize_pad_images(test_folder, target_size, padding_color)

print("Processing valid folder...")
resize_pad_images(valid_folder, target_size, padding_color)

Processing train folder...
Processing folder: ../data/train/Aphids
Processing image: ../data/train/Aphids\00_aphids_train.jpg
Saved image: ../data/train/Aphids\00_aphids_train.jpg
Processing image: ../data/train/Aphids\01_aphids_train.jpg
Saved image: ../data/train/Aphids\01_aphids_train.jpg
Processing image: ../data/train/Aphids\02_aphids_train.jpg
Saved image: ../data/train/Aphids\02_aphids_train.jpg
Processing image: ../data/train/Aphids\03_aphids_train.jpg
Saved image: ../data/train/Aphids\03_aphids_train.jpg
Processing image: ../data/train/Aphids\04_aphids_train.jpg
Saved image: ../data/train/Aphids\04_aphids_train.jpg
Processing image: ../data/train/Aphids\05_aphids_train.jpg
Saved image: ../data/train/Aphids\05_aphids_train.jpg
Processing image: ../data/train/Aphids\06_aphids_train.jpg
Saved image: ../data/train/Aphids\06_aphids_train.jpg
Processing image: ../data/train/Aphids\07_aphids_train.jpg
Saved image: ../data/train/Aphids\07_aphids_train.jpg
Processing image: ../data/tra

In [21]:
# folder names containing images of the things you want to classify
disease_classes = [
    'Aphids',
    'Botrytis',
    'Dehydration',
    'Healthy',
    'Leaf Miners',
    'Nitrogen Deficiency',
    'Nutrient Burn',
    'Overwatering',
    'PH Fluctuation',
    'Phosphorus Deficiency',
    'Potassium Deficiency',
    'Powdery Mildew',
    'Septoria']

# Data Augmentation

### Random rotation between -40 to +40 degrees, random zoom in/out by 20%, random brightness between 80% and 120% were applied as data augmentation.

In [19]:
# define an image data generator
# Data augmentation: Applies random distortions and transformations to the images (only on your training data!).

data_gen = image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    # preprocessing_function=preprocess_input,
    fill_mode='nearest',
    rotation_range=40,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2]
)

In [26]:
from keras.preprocessing.image import img_to_array, load_img, array_to_img

In [22]:
# Function to apply augmentation and save images
def augment_images_in_folder(folder_path, target_num_images, datagen):
    for class_folder in os.listdir(folder_path):
        class_folder_path = os.path.join(folder_path, class_folder)

        if os.path.isdir(class_folder_path):
            print(f"Processing folder: {class_folder_path}")
            
            # Get all images in the class folder
            image_files = [f for f in os.listdir(class_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            num_existing_images = len(image_files)
            
            # Calculate how many images need to be generated
            num_to_generate = target_num_images - num_existing_images
            
            if num_to_generate > 0:
                print(f"Generating {num_to_generate} images for class {class_folder}")

                # Loop over the existing images and create augmented images
                generated = 0
                while generated < num_to_generate:
                    for img_name in image_files:
                        img_path = os.path.join(class_folder_path, img_name)
                        
                        # Load image and convert to array
                        img = load_img(img_path)
                        img_array = img_to_array(img)
                        img_array = np.expand_dims(img_array, axis=0)

                        # Generate a batch of augmented images
                        aug_iter = datagen.flow(img_array, batch_size=1)

                        # Save the augmented image with _aug appended to the filename
                        aug_img = next(aug_iter)[0].astype('uint8')
                        aug_img_pil = array_to_img(aug_img)

                        # Create new filename for augmented image
                        new_img_name = f"{os.path.splitext(img_name)[0]}_aug{generated}.jpg"
                        new_img_path = os.path.join(class_folder_path, new_img_name)

                        # Save the augmented image
                        aug_img_pil.save(new_img_path)
                        print(f"Saved augmented image: {new_img_path}")

                        generated += 1
                        if generated >= num_to_generate:
                            break

In [24]:
# Define target number of images per class
train_target_num_images = 100
valid_target_num_images = 20

In [33]:
# Apply augmentation to the train folder
print("Processing train folder...")
augment_images_in_folder(train_folder, train_target_num_images, data_gen)


Processing train folder...
Processing folder: ../data/train/Aphids
Generating 90 images for class Aphids
Saved augmented image: ../data/train/Aphids\00_aphids_train_aug0.jpg
Saved augmented image: ../data/train/Aphids\01_aphids_train_aug1.jpg
Saved augmented image: ../data/train/Aphids\02_aphids_train_aug2.jpg
Saved augmented image: ../data/train/Aphids\03_aphids_train_aug3.jpg
Saved augmented image: ../data/train/Aphids\04_aphids_train_aug4.jpg
Saved augmented image: ../data/train/Aphids\05_aphids_train_aug5.jpg
Saved augmented image: ../data/train/Aphids\06_aphids_train_aug6.jpg
Saved augmented image: ../data/train/Aphids\07_aphids_train_aug7.jpg
Saved augmented image: ../data/train/Aphids\08_aphids_train_aug8.jpg
Saved augmented image: ../data/train/Aphids\09_aphids_train_aug9.jpg
Saved augmented image: ../data/train/Aphids\00_aphids_train_aug10.jpg
Saved augmented image: ../data/train/Aphids\01_aphids_train_aug11.jpg
Saved augmented image: ../data/train/Aphids\02_aphids_train_aug12

In [27]:
# Apply augmentation to the train folder
print("Processing valid folder...")
augment_images_in_folder(valid_folder, valid_target_num_images, data_gen)


Processing valid folder...
Processing folder: ../data/valid/Aphids
Generating 18 images for class Aphids
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug0.jpg
Saved augmented image: ../data/valid/Aphids\01_aphids_valid_aug1.jpg
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug2.jpg
Saved augmented image: ../data/valid/Aphids\01_aphids_valid_aug3.jpg
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug4.jpg
Saved augmented image: ../data/valid/Aphids\01_aphids_valid_aug5.jpg
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug6.jpg
Saved augmented image: ../data/valid/Aphids\01_aphids_valid_aug7.jpg
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug8.jpg
Saved augmented image: ../data/valid/Aphids\01_aphids_valid_aug9.jpg
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug10.jpg
Saved augmented image: ../data/valid/Aphids\01_aphids_valid_aug11.jpg
Saved augmented image: ../data/valid/Aphids\00_aphids_valid_aug12

In [35]:
print(disease_classes)

['Aphids', 'Botrytis', 'Dehydration', 'Healthy', 'Leaf Miners', 'Nitrogen Deficiency', 'Nutrient Burn', 'Overwatering', 'PH Fluctuation', 'Phosphorus Deficiency', 'Potassium Deficiency', 'Powdery Mildew', 'Septoria']


In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(416, 416),  # Resize images
    batch_size=32,
    class_mode='categorical',      # Use 'sparse' for integer labels
    shuffle=True              # Shuffle the data
)

valid_generator = valid_datagen.flow_from_directory(
    valid_folder,
    target_size=(416, 416),
    batch_size=32,
    class_mode='categorical',
    shuffle=False    
)


Found 1300 images belonging to 13 classes.
Found 260 images belonging to 13 classes.


In [32]:
# Print class indices
print("Class indices for training data:")
print(train_generator.class_indices)

Class indices for training data:
{'Aphids': 0, 'Botrytis': 1, 'Dehydration': 2, 'Healthy': 3, 'Leaf Miners': 4, 'Nitrogen Deficiency': 5, 'Nutrient Burn': 6, 'Overwatering': 7, 'PH Fluctuation': 8, 'Phosphorus Deficiency': 9, 'Potassium Deficiency': 10, 'Powdery Mildew': 11, 'Septoria': 12}
